
Import Statement

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt



Create the network as in section 1

In [2]:
df= pd.read_csv('datasets\dataset.csv')
df=df.sort_values(by=['user_id','date_created'])
df["book_chapter"] = df["chapter"].astype(str) + df["book"].astype(str)
df["source"] =df["book_chapter"]
df["target"]= df["source"].shift(-1)  
df["user_check"]= df["user_id"].shift(-1)
df['keep_drop'] = np.where((df['user_check'] == df['user_id'])
                     , df['user_id'], np.nan)
df=df.dropna(subset=["keep_drop"])
G=nx.from_pandas_edgelist(df,"source",'target', create_using=nx.DiGraph)

Calculate the centrality measure for the nodes

In [3]:
out=nx.out_degree_centrality(G)
nx.set_node_attributes(G, out, 'out-degree')

In [4]:
bb = nx.betweenness_centrality(G)
nx.set_node_attributes(G, bb, 'betweenness')


In [5]:
eigen= nx.eigenvector_centrality(G)
nx.set_node_attributes(G, eigen, 'eigen')

Create a list of all the chapter and all the centrality

In [6]:
data_list=list(G.nodes(data=True))

Convert the list into a dictionary with chapter as the key and the centrality measures as values.

In [7]:
data={}
data['book_chapter']=[x[0] for x in data_list]
data['out-degree'] = [x[1]['out-degree'] for x in data_list]
data['eigen'] = [x[1]['eigen'] for x in data_list]
data['betweenness'] = [x[1]['betweenness'] for x in data_list]
df1 = pd.DataFrame(data)
df1

,book_chapter,out-degree,eigen,betweenness
0,chapter-11clinton3,0.001004,1.814720e-28,0.000000
1,chapter-12clinton3,0.001004,7.633365e-04,0.000822
2,chapter-3clinton3,0.002007,1.244880e-02,0.002502
3,chapter-6clinton3,0.001004,3.082603e-03,0.000377
4,about-this-bookbjarnason,0.002007,3.560926e-03,0.008865
...,...,...,...,...
1989,chapter-3sholmes2,0.000502,9.223888e-04,0.000000
1990,chapter-12sholmes2,0.000502,2.738893e-04,0.000378
1991,appendix-bsholmes2,0.000000,5.438649e-05,0.000000
1992,about-the-authorsnickoloff2,0.000502,7.732873e-03,0.000000


Calculate what chapter are connected based on the adjacency matrices of a single chapter

In [8]:
row1=[n for n in G['chapter-11clinton3']]

In [9]:
df2= pd.DataFrame(row1)

Join the dataframes by chapter

In [10]:
df3=df2.set_index(0).join(df1.set_index('book_chapter'))

Sort the values by eigenvector in descending order (largest first)

In [11]:
df3=df3.sort_values(by=['eigen'],ascending=False)


In [12]:
df3

,out-degree,eigen,betweenness
0,,,
chapter-12clinton3,0.001004,7.633365e-04,0.000822
chapter-11clinton3,0.001004,1.814720e-28,0.000000
